<div style="background: linear-gradient(135deg, #034694 0%, #1E8449 50%, #D4AC0D 100%); color: white; padding: 20px; border-radius: 10px; box-shadow: 0 4px 8px rgba(0,0,0,0.2);">
    <h1 style="color: #FFF; text-shadow: 1px 1px 3px rgba(0,0,0,0.5);">💬 | Step 2: Customize The Tone & Style With SFT </h1>
        <p style="font-size: 16px; line-height: 1.6;">
            We used few shot examples to prompt-engineer a better tone. We used RAG to ground responses in our data. But this keeps growing our prompt lengths (increasing token costs and reduce effective context window available for output). How can we improve the tone and style of our bot with _more examples_ and shorter prompt length?
        </p>
</div>

<div style="display: flex; align-items: center; justify-content: left; padding: 5px; height: 40px; background: linear-gradient(90deg, #333333 0%, #777777 50%, #BBBBBB 100%); border-radius: 8px; box-shadow: 0 2px 8px rgba(0,0,0,0.12); font-size: 1.5em; font-weight: bold; color: #fff;">
    Step 1 : Understand Zava Scenarios
</div>
<br/>
<div style="display: flex; align-items: center; justify-content: left; padding: 5px; height: 40px; background: linear-gradient(90deg, #7873f5 0%, #ff6ec4 100%); border-radius: 8px; box-shadow: 0 2px 8px rgba(0,0,0,0.12); font-size: 1.5em; font-weight: bold; color: #fff;">
    Step 3: Be More Cost-Effective With Distillation
</div>
<br/>

---
### 1. Check Environment Variables

In [1]:
import os

openai_key = os.getenv("AZURE_OPENAI_API_KEY")
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
model_name = "gpt-4.1"
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2025-02-01-preview")

if not openai_key or not openai_endpoint:
    print("Error: Missing AZURE_OPENAI_KEY or AZURE_OPENAI_ENDPOINT environment variable.")

print("Using Model:", model_name)
print("Using API Version:", api_version)

Using Model: gpt-4.1
Using API Version: 2025-02-01-preview


---
### 2. Validate Training Dataset

In [2]:
# Identify Training and Validation datafiles


training_file = "../data/basic_sft_training.jsonl" 
validation_file = "../data/basic_sft_validation.jsonl"

In [3]:
# Run preliminary checks

import json

# Load the training set
with open(training_file, 'r', encoding='utf-8') as f:
    training_dataset = [json.loads(line) for line in f]

# Training dataset stats
print("Number of examples in training set:", len(training_dataset))
print("First example in training set:")
for message in training_dataset[0]["messages"]:
    print(message)

# Load the validation set
with open(validation_file, 'r', encoding='utf-8') as f:
    validation_dataset = [json.loads(line) for line in f]

# Validation dataset stats
print("\nNumber of examples in validation set:", len(validation_dataset))
print("First example in validation set:")
for message in validation_dataset[0]["messages"]:
    print(message)

Number of examples in training set: 40
First example in training set:
{'role': 'system', 'content': 'You are Cora, a polite, factual and helpful assistant for Zava, a DIY hardware store.'}
{'role': 'user', 'content': 'Is water-based polyurethane better than oil-based?'}
{'role': 'assistant', 'content': '🤔 Both have benefits! Water-based at $43 dries faster, oil-based at $34 gives amber tone. Which matters most?'}

Number of examples in validation set: 10
First example in validation set:
{'role': 'system', 'content': 'You are Cora, a polite, factual and helpful assistant for Zava, a DIY hardware store.'}
{'role': 'user', 'content': "What's good for painting wrought iron?"}
{'role': 'assistant', 'content': '🔧 Metal Primer is perfect for wrought iron! Metal Primer at $36 first, then any topcoat. Existing rust present?'}


---
### 3. Assess Token Counts For Data

In [4]:
# Validate token counts

import json
import tiktoken
import numpy as np

encoding = tiktoken.get_encoding("o200k_base") # default encoding for gpt-4o models. This requires the latest version of tiktoken to be installed.

def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

files = [training_file, validation_file]

for file in files:
    print(f"Processing file: {file}")
    with open(file, 'r', encoding='utf-8') as f:
        dataset = [json.loads(line) for line in f]

    total_tokens = []
    assistant_tokens = []

    for ex in dataset:
        messages = ex.get("messages", {})
        total_tokens.append(num_tokens_from_messages(messages))
        assistant_tokens.append(num_assistant_tokens_from_messages(messages))

    print_distribution(total_tokens, "total tokens")
    print_distribution(assistant_tokens, "assistant tokens")
    print('*' * 50)

Processing file: ../data/basic_sft_training.jsonl

#### Distribution of total tokens:
min / max: 62, 78
mean / median: 68.375, 68.0
p5 / p95: 65.0, 71.1

#### Distribution of assistant tokens:
min / max: 19, 30
mean / median: 23.75, 24.0
p5 / p95: 20.9, 26.1
**************************************************
Processing file: ../data/basic_sft_validation.jsonl

#### Distribution of total tokens:
min / max: 66, 76
mean / median: 69.4, 69.0
p5 / p95: 66.9, 73.3

#### Distribution of assistant tokens:
min / max: 22, 28
mean / median: 24.4, 24.0
p5 / p95: 22.9, 26.2
**************************************************


---
### 4. Upload Fine-Tuning Data To Cloud

In [5]:
# Create Azure OpenAI Client

import os
from openai import AzureOpenAI

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),
  api_version = os.getenv("AZURE_OPENAI_API_VERSION")
)

In [6]:
# Upload the training and validation dataset files to Azure OpenAI with the SDK.
# Note: You can visit the Azure AI Foundry Portal - and look under your Azure AI Project's 'Data Files' tab to see the uploaded files.

training_response = client.files.create(
    file = open(training_file, "rb"), purpose="fine-tune"
)
training_file_id = training_response.id

validation_response = client.files.create(
    file = open(validation_file, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-df310fefd1cc48148327305ec11e246a
Validation file ID: file-0ad0d42ef65f4865abba1189098f7bca


---
### 5. Submit The Fine-Tuning Job

In [7]:
# Submit fine-tuning training job
# Note that the model you specify here must be one that can be fine-tuned.
# Currently gpt-4.1 can be fine-tuned only in Sweden Central and North Central US
# See: https://learn.microsoft.com/en-us/azure/ai-foundry/openai/how-to/fine-tuning?tabs=azure-openai&pivots=programming-language-python#prerequisites
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-4.1", # Enter base model name. Note that in Azure OpenAI the model name contains dashes and cannot contain dot/period characters.
    seed = 105,  # seed parameter controls reproducibility of the fine-tuning job. If no seed is specified one will be generated automatically.
    hyperparameters={
        "learning_rate_multiplier": 2.0,  # Higher LR for faster training
        "n_epochs": 3,  # Reduced epochs (default is often overkill)
        "batch_size": 16,  # Larger batch for stability
    }
)

job_id = response.id

# You can use the job ID to monitor the status of the fine-tuning job.
# The fine-tuning job will take some time to start and complete.
print("Job ID:", response.id)
print("Status:", response.id)
print(response.model_dump_json(indent=2))

BadRequestError: Error code: 400 - {'error': {'code': 'invalidPayload', 'message': 'The specified file reference must point to a completed file import.'}}

---

### 6. Track Fine-Tuning Job Status

In [ ]:
# Track training status

from IPython.display import clear_output
import time

start_time = time.time()

# Get the status of our fine-tuning job.
response = client.fine_tuning.jobs.retrieve(job_id)

status = response.status

# If the job isn't done yet, poll it every 10 seconds.
while status not in ["succeeded", "failed"]:
    time.sleep(10)

    response = client.fine_tuning.jobs.retrieve(job_id)
    print(response.model_dump_json(indent=2))
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = response.status
    print(f'Status: {status}')
    clear_output(wait=True)

print(f'Fine-tuning job {job_id} finished with status: {status}')

# List all fine-tuning jobs for this resource.
print('Checking other fine-tune jobs for this resource.')
response = client.fine_tuning.jobs.list()
print(f'Found {len(response.data)} fine-tune jobs.')

Fine-tuning job ftjob-c387da6dc41344a48a2c00cdb543f9ff finished with status: succeeded
Checking other fine-tune jobs for this resource.
Found 20 fine-tune jobs.


---

### 7. List Fine-Tuning Events

In [ ]:
response = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=10)
print(response.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "ftevent-ddad2e8381574b9bb5115a949deade7d",
      "created_at": 1759398914,
      "level": "info",
      "message": "Training tokens billed: 8000",
      "object": "fine_tuning.job.event",
      "data": null,
      "type": "message"
    },
    {
      "id": "ftevent-086c7c89dc9a45ddb76150dcdd983c36",
      "created_at": 1759398914,
      "level": "info",
      "message": "Completed results file: file-963292befa624ed2ade8c587609042bb",
      "object": "fine_tuning.job.event",
      "data": null,
      "type": "message"
    },
    {
      "id": "ftevent-b40ea3b38f5e483b840fdb429334b298",
      "created_at": 1759398911,
      "level": "info",
      "message": "Model Evaluation Passed.",
      "object": "fine_tuning.job.event",
      "data": null,
      "type": "message"
    },
    {
      "id": "ftevent-276e0b962fed4fd6b500d3e2a2754355",
      "created_at": 1759398854,
      "level": "info",
      "message": "Job succeeded.",
      "object": "fine_tuning.jo

---

### 8. List Fine-Tuning Checkpoints

In [ ]:
response = client.fine_tuning.jobs.checkpoints.list(job_id)
print(response.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "ftchkpt-e30cc21551b24afe8e4a8510522ec286",
      "created_at": 1759396725,
      "fine_tuned_model_checkpoint": "gpt-4.1-2025-04-14.ft-c387da6dc41344a48a2c00cdb543f9ff",
      "fine_tuning_job_id": "ftjob-c387da6dc41344a48a2c00cdb543f9ff",
      "metrics": {
        "full_valid_loss": 4.2378693927418105,
        "full_valid_mean_token_accuracy": 0.375,
        "step": 8.0,
        "train_loss": 4.689654350280762,
        "train_mean_token_accuracy": 0.3235294222831726,
        "valid_loss": 4.310887895311628,
        "valid_mean_token_accuracy": 0.3595238095238095
      },
      "object": "fine_tuning.job.checkpoint",
      "step_number": 8
    },
    {
      "id": "ftchkpt-3027a2f97d01454f87de5f5a561fda4d",
      "created_at": 1759396643,
      "fine_tuned_model_checkpoint": "gpt-4.1-2025-04-14.ft-c387da6dc41344a48a2c00cdb543f9ff:ckpt-step-6",
      "fine_tuning_job_id": "ftjob-c387da6dc41344a48a2c00cdb543f9ff",
      "metrics": {
        "full_valid_l

---

### 9. Retrieve Fine-Tuned Model Name

In [ ]:
# Retrieve fine_tuned_model name

response = client.fine_tuning.jobs.retrieve(job_id)

print(response.model_dump_json(indent=2))
fine_tuned_model = response.fine_tuned_model

{
  "id": "ftjob-c387da6dc41344a48a2c00cdb543f9ff",
  "created_at": 1759394308,
  "error": null,
  "fine_tuned_model": "gpt-4.1-2025-04-14.ft-c387da6dc41344a48a2c00cdb543f9ff",
  "finished_at": 1759398914,
  "hyperparameters": {
    "batch_size": 16,
    "learning_rate_multiplier": 2.0,
    "n_epochs": 3
  },
  "model": "gpt-4.1-2025-04-14",
  "object": "fine_tuning.job",
  "organization_id": null,
  "result_files": [
    "file-963292befa624ed2ade8c587609042bb"
  ],
  "seed": 105,
  "status": "succeeded",
  "trained_tokens": 10485,
  "training_file": "file-7fae087b7ad3415a976c647af3a074bd",
  "validation_file": "file-3eaf80c42c344ff5a7a7bd992fc1de15",
  "estimated_finish": 1759395748,
  "integrations": null,
  "metadata": null,
  "method": null
}


---

### 10. Deploy Fine-Tuned Model For Testing

For now - we deployed this manually from the Azure AI Foundry Portal - using the **developer tier** which allows us to test our fine-tuned model for the cost of just inferencing. Once we deploy it, we can try it out

> Prompt 1: What kind of paint should I buy for my outdoor deck?

```txt
🪵 Deck protection options! Semi-Transparent Deck Stain at 38 enhances wood grain, or Deck & Fence Stain at 36 for UV protection?
```

> Prompt 2: I'm painting over rust - what spray paint should I use?

👍 Right choice! Rust Prevention Spray at $13 applies directly over rust with long-lasting protection. Primer recommendation?

**Insights**

In both the examples above we can note that the response now accurately follows our Zava guidelines for "polite, factual and helpful"
- Every response starts with an emoji
- The first sentence is always an acknowledgement of the user ("polite")
- The next sentence is always an informative segment ("factual")
- The final senteance is always an offer to follow up ("helpful")

And note that we have the succinct responses we were looking for _without adding few-shot examples_, making the prompts shorter and thus saving both token costs and processing latency.

---
### Teardown

Once you are done with this lab, don't forget to tear down the infrastructure. The developer tier model will be torn down automatically (after 24 hours?) but it is better to proactively delete the resource group and release all model quota.


<div style="display: flex; align-items: center; justify-content: left; padding: 5px; height: 40px; background: linear-gradient(90deg, #7873f5 0%, #ff6ec4 100%); border-radius: 8px; box-shadow: 0 2px 8px rgba(0,0,0,0.12); font-size: 1.5em; font-weight: bold; color: #fff;">
    Next: Be More Cost-Effective With Distillation
</div>